# Nachbarschaft nach dem Voxelfilter

In [1]:
import pdal 
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
from scipy.spatial import KDTree
import os
import json
import time 

from interessant import * # Bei Änderungen Kernel neu starten

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
run = run24
#run = run14
# filename = interessant['OLA gleiche Höhe wie Gleis']

# Bahnsteig: 29; Gleis hohe Intensität: 11; Weiche B: 16; Unterirdischer Bhf: 20; Gleis weit abseits: 23; Betondeckel: 28; Zug run 14 A (in run24 Achszähler): 6; 
# Viele Gleise: 33; Anfang Weiche: 34; OLA gleiche H: 35; Y: 37
key = list(interessant.keys())[38] 
filename = interessant[key]
print(key, filename)

filename = os.path.join(run, filename)
if not os.path.exists(filename):
    raise FileNotFoundError(filename)

Weiche C 4473850_5336225.copc.laz


In [3]:
thresh = 8  # z.B. 5 oder 8
majority_tresh  = 0.5 # Erster Durchgang 0.3, bei "Gleis hohe Intensität" gibt 0.5 ein viel besseres Ergebnis

voxel_size = 1.0

voxel_size = 25 / 30
print("Voxel size:", voxel_size)

minimum_points = 50 # Erste Versuche mit 100, aber viel schwarz bei abseits liegenden Gleisen. 50 ist besser.

intensity_threshold = 14500
downsample_radius = 0.3
neighborhood_radius = 0.5

Voxel size: 0.8333333333333334


In [4]:
filename 

'/media/riannek/minimax/gleis/2024-08-13/01/run24/01/4473850_5336225.copc.laz'

In [5]:
import subprocess
#subprocess.Popen(["pyvistaviewer", filename])

## Voxelfilter

In [6]:
start = time.time()

pipeline = pdal.Pipeline([pdal.Reader(filename)])
pipeline.execute()
points = pipeline.arrays[0]
print("Time", time.time() - start)
print("Number of points:", len(points))

Time 0.8098118305206299
Number of points: 1598139


In [7]:
'X' in points.dtype.names

True

In [8]:
xyz = np.vstack((points['X'], points['Y'], points['Z'])).transpose()

In [9]:
xyz.shape[0]

1598139

In [10]:
# Offset entfernen (aber gerundet, damit Kachelgrenzen ganze Zahlen bleiben)
offset = xyz.mean(axis=0).round() 
# xyz -= offset   # Nur für Visualisierung benötigt

In [11]:
points['Classification'] = 0 # Unclassified
RAIL = 20

In [12]:
maxp = xyz.max(axis=0)
minp = xyz.min(axis=0)
maxp, minp

(array([4.47387500e+06, 5.33625000e+06, 5.28954453e+02]),
 array([4.47385000e+06, 5.33622500e+06, 5.14541453e+02]))

In [13]:
voxels = xyz.copy()
voxels[:, :2] = ((xyz[:, :2] - minp[:2]) // voxel_size).astype(int)

In [14]:
# Anzahl der Voxel checken
np.ceil((maxp[:2] - minp[:2]) / voxel_size).astype(int)

array([30, 30])

In [15]:
from collections import defaultdict
voxel_dict = defaultdict(list)
index_dict = defaultdict(list)

# Füllen des Dictionaries
for idx, (point, voxel) in enumerate(zip(xyz, voxels)):
    voxel_key = tuple(voxel[:2])
    voxel_dict[voxel_key].append(point[2])
    index_dict[voxel_key].append(idx)

In [16]:
for key, z_values in voxel_dict.items():
    
    # Threshold on number of points in voxel
    if len(z_values) < minimum_points:
        continue

    indices = np.array(index_dict[key])
    z_values = np.array(z_values)
    ground_level = np.percentile(z_values, 10) # 10% Percentile
    # Check that there are almost no points 0.5 to 4.5 m above the ground
    # But allow for some noise
    # thresh = 3 # Der einfachheit halber oben
    count = ((z_values > ground_level + 0.5) & (z_values < ground_level + 4.5)).sum()

    if count <= thresh:
        # Look for points within 0.5 m above ground and get 98% percentile ODER 99.5
        mask = (z_values > ground_level) & (z_values < ground_level + 0.5)
        try:
            candidates_top = np.percentile(z_values[mask], 99.5)
        except IndexError:
            # Fails if there are no points in the masked array
            continue

        # Oude Elberink require the height difference > 0.1 m
        # And mark only the points 10 cm below the top as rail point candidates
        if candidates_top - ground_level > 0.1:
            mask = (z_values > candidates_top - 0.1) & (z_values < candidates_top + 0.05)

            # Also make sure these are only a minority of the points (otherwise it's a slope)
            if mask.sum() < majority_tresh * len(z_values):  # z.B. 0.3
                points['Classification'][indices[mask]] = RAIL


In [17]:
candidates = points[points["Classification"] == RAIL]
candidates.shape

(90623,)

In [18]:
1598139 / 90623

17.63502642816945

## Noise Filter

In [19]:
# filters.outlier sets Classification to 7, filters.range removes the points with Classification 7

noise_filter = pdal.Filter("filters.outlier", method="statistical", mean_k=10, multiplier=2.0).pipeline(candidates) | pdal.Filter("filters.range", limits="Classification![7:7]")
print(noise_filter.toJSON())
noise_filter.execute()
candidates = noise_filter.arrays[0]
candidates.shape 

[{"type": "filters.outlier", "method": "statistical", "mean_k": 10, "multiplier": 2.0, "tag": "filters_outlier1"}, {"type": "filters.range", "limits": "Classification![7:7]", "tag": "filters_range1"}]


(88499,)

## Schreiben und lesen testen

In [20]:
start = time.time()
out_pipeline = pdal.Writer("zeittest.laz").pipeline(candidates[['X', 'Y', 'Z', 'Intensity']])
out_pipeline.execute()
print("Time:", time.time() - start)

Time: 0.04015755653381348


In [21]:
start = time.time()
pipeline = pdal.Pipeline([pdal.Reader("zeittest.laz")])
pipeline.execute()
test = pipeline.arrays[0]
print("Time:", time.time() - start)
test

Time: 0.054163217544555664


array([(4473864.08, 5336235.15, 516.73, 21588, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0., 0, 0, 0., 0, 0, 0, 0),
       (4473862.87, 5336231.53, 516.89, 15934, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0., 0, 0, 0., 0, 0, 0, 0),
       (4473863.09, 5336235.11, 516.78, 20046, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0., 0, 0, 0., 0, 0, 0, 0),
       ...,
       (4473857.03, 5336245.62, 516.59, 21074, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0., 0, 0, 0., 0, 0, 0, 0),
       (4473859.89, 5336247.86, 516.82, 23130, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0., 0, 0, 0., 0, 0, 0, 0),
       (4473859.76, 5336246.18, 516.59, 19789, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0., 0, 0, 0., 0, 0, 0, 0)],
      dtype=[('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2'), ('ReturnNumber', 'u1'), ('NumberOfReturns', 'u1'), ('ScanDirectionFlag', 'u1'), ('EdgeOfFlightLine', 'u1'), ('Classification', 'u1'), ('Synthetic', 'u1'), ('KeyPoint', 'u1'), ('Withheld', 'u1'), ('Overlap', 'u1'), ('ScanAngleRank', '<f4'), ('UserData', 'u1'), ('PointSourceId', '<u2'), ('GpsTime', '<f8')

## View Settings

In [22]:
# Viewsettings mit strg + c kopieren und hier einfügen

viewsettings = '''
{
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : false,
	"trajectory" : 
	[
		{
			"boundingbox_max" : [ 11.999975427985191, 11.99998692702502, 13.124079998226534 ],
			"boundingbox_min" : [ -13.000024572014809, -13.00001307297498, -3.9965200017734333 ],
			"field_of_view" : 60.0,
			"front" : [ -0.20468464372193082, -0.82045900926496551, 0.53380821531742795 ],
			"lookat" : [ -2.1145501200370735, -2.6052610037108783, 1.4494799802055294 ],
			"up" : [ 0.19010212482081987, 0.50164960558000959, 0.84392467398461002 ],
			"zoom" : 0.55999999999999983
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}

'''

viewsettings = json.loads(viewsettings)

front = viewsettings["trajectory"][0]["front"]
lookat = viewsettings["trajectory"][0]["lookat"]
up = viewsettings["trajectory"][0]["up"]
zoom = viewsettings["trajectory"][0]["zoom"]

## Candidate and Seed Points

In [23]:
xyz = np.vstack((candidates['X'], candidates['Y'], candidates['Z'])).transpose()
xyz -= offset

In [24]:
pcd_candidates = o3d.geometry.PointCloud()
pcd_candidates.points = o3d.utility.Vector3dVector(xyz)
pcd_candidates.paint_uniform_color([0.5, 0.5, 0.5])

PointCloud with 88499 points.

In [25]:
low_intensity = candidates[candidates["Intensity"] < intensity_threshold]
low_intensity.shape

(63156,)

In [26]:
xyz_low = np.vstack((low_intensity['X'], low_intensity['Y'], low_intensity['Z'])).transpose()
xyz_low -= offset

pcd_low_intensity = o3d.geometry.PointCloud()
pcd_low_intensity.points = o3d.utility.Vector3dVector(xyz_low)
pcd_low_intensity.paint_uniform_color([0, 0, 0.7])

PointCloud with 63156 points.

In [27]:
# Downsample with poisson sampling

downsampling_pipeline = pdal.Filter("filters.sample", radius=downsample_radius).pipeline(low_intensity)
downsampling_pipeline.execute()
seed_points = downsampling_pipeline.arrays[0]
seed_points.shape 

(362,)

In [28]:
xyz_seed = np.vstack((seed_points['X'], seed_points['Y'], seed_points['Z'])).transpose()
xyz_seed -= offset

pcd_seed_points = o3d.geometry.PointCloud()
pcd_seed_points.points = o3d.utility.Vector3dVector(xyz_seed)
pcd_seed_points.paint_uniform_color([1, 0, 0])

PointCloud with 362 points.

In [29]:
o3d.visualization.draw_geometries([
    pcd_candidates, 
    pcd_low_intensity, 
    pcd_seed_points
    ], front=front, lookat=lookat, up=up, zoom=zoom)

In [30]:
o3d.visualization.draw_geometries([pcd_seed_points], front=front, lookat=lookat, up=up, zoom=zoom)

## Anzahl Punkte in Nachbarschaft

In [31]:
# k-D tree with all candidate points
tree = KDTree(xyz)  

In [32]:
# indices: ndarray (dtype object) with a list of indices for each seed point
indices = tree.query_ball_point(xyz_seed, r=neighborhood_radius)

In [33]:
# Use pyvista to get scalar colors with color bar
import pyvista as pv
pcd_hood = pv.PolyData(xyz_seed)

In [34]:
seed_point_count = xyz_seed.shape[0]

In [35]:
# Count points in neighborhood
pcd_hood['counts'] = np.array([[len(lst) for lst in indices]]).T

In [ ]:
pv.plot(pcd_hood, scalars='counts', 
        render_points_as_spheres=True, point_size=10,
        show_scalar_bar=True,
        )

Widget(value='<iframe src="http://localhost:39901/index.html?ui=P_0x7f44d1ff7790_0&reconnect=auto" class="pyvi…

 JS Error => error: Uncaught TypeError: Cannot mix BigInt and other types, use explicit conversions
 JS Error => error: Uncaught TypeError: Cannot mix BigInt and other types, use explicit conversions


In [37]:
# To find a threshold for low number of points, create a clipped version
pcd_hood['counts clipped'] = np.clip(pcd_hood['counts'], 0, 100)

In [38]:
# pv.plot(pcd_hood, scalars='counts clipped', 
#         render_points_as_spheres=True, point_size=10,
#         show_scalar_bar=True,
#         )

Bei weit abseits liegenden Gleisen zum Teil extrem wenige Punkte in der hood, zwischen 10 und 20

In [39]:
# COUNT

p = pv.Plotter()
p.add_mesh_threshold(pcd_hood, 'counts clipped', title="Counts (Clipped)", all_scalars=True, render_points_as_spheres=True, point_size=10)
p.show()

Widget(value='<iframe src="http://localhost:39901/index.html?ui=P_0x7f444dbe3190_1&reconnect=auto" class="pyvi…

In [40]:
def pca(cloud):
    """Use PCA to get einvalues and eigenvectors of a point cloud"""
    mean = np.mean(cloud, axis=0)
    centered = cloud - mean
    cov_matrix = np.cov(centered, rowvar=False) # row variance nicht berechnen
    eigenvals, eigenvecs = np.linalg.eig(cov_matrix)
    sorted_indices = np.argsort(eigenvals)[::-1]
    sorted_eigenvals = eigenvals[sorted_indices]
    sorted_eigenvecs = eigenvecs[:,sorted_indices]
    # Returned vectors are in columns, first vector is eigenvec[:, 0] == eigenvec.T[0]
    return sorted_eigenvals, sorted_eigenvecs

def linearity(eigenvals):
    """Calculate the linearity of a point cloud"""
    return (eigenvals[0] - eigenvals[1]) / eigenvals[0]

In [41]:
def theta(eigenvects):
    """Angle between the first eigenvector and the z-axis"""
    cos_theta = eigenvects.T[0] @ np.array([0, 0, 1]) / np.linalg.norm(eigenvects[0])
    return np.arccos(cos_theta) * 180 / np.pi

In [42]:
linearity_at_seed = np.empty((seed_point_count,1), dtype=float)
theta_at_seed = np.empty((seed_point_count,1), dtype=float)
first_vec = np.zeros((seed_point_count,3), dtype=float)

In [43]:
for i in range(seed_point_count):
    hood = xyz[indices[i]]
    if hood.shape[0] > 20:   # Was macht hier Sinn? 10, 20?
        eigenvals, eigenvects = pca(hood)
        linearity_at_seed[i] = linearity(eigenvals)
        theta_at_seed[i] = theta(eigenvects)
        first_vec[i] = eigenvects.T[0]
    else:
        linearity_at_seed[i] = np.nan
        theta_at_seed[i] = np.nan

In [44]:
xyz_seed  

array([[ 2.47368982, -4.07180798, -1.16954745],
       [-0.50691018, -4.77530798, -1.29294745],
       [-0.79361018, -3.40320798, -1.28504745],
       ...,
       [-2.82281018,  8.86789202, -1.37274745],
       [ 3.86978982,  7.12529202, -1.26164745],
       [ 1.52108982, 12.64679202, -1.44274745]])

In [45]:
first_vec 

array([[ 0.8681293 ,  0.4898326 ,  0.08009714],
       [ 0.18343781, -0.98240886, -0.03497726],
       [ 0.13875129, -0.9902859 ,  0.00905079],
       ...,
       [ 0.12254469, -0.99245198,  0.00467592],
       [ 0.98966892,  0.11867446,  0.0804475 ],
       [ 0.99821676, -0.05931359,  0.0067229 ]])

In [46]:
pcd_hood['theta'] = theta_at_seed

pv.plot(pcd_hood, scalars='theta', 
        render_points_as_spheres=True, point_size=10,
        show_scalar_bar=True,
        )

Widget(value='<iframe src="http://localhost:39901/index.html?ui=P_0x7f44d1ff7a00_2&reconnect=auto" class="pyvi…

In [47]:
pcd_hood['linearity'] = linearity_at_seed 

# pv.plot(pcd_hood, scalars='linearity', 
#         render_points_as_spheres=True, point_size=10,
#         show_scalar_bar=True,
#         )

kononenFullyAutomatedExtraction2024 verwenden threshold 0.98 (behalten aber zusätzlich auch Punkte in Nachbarschaften mit hoher Punktdichte)

In [48]:
p = pv.Plotter()


# Add lines (SLOW)
first_vec_scaled = 0.5 * first_vec
for i in range(len(xyz_seed)):
    start_point = xyz_seed[i]
    end_point = start_point + first_vec_scaled[i]
    line = pv.Line(start_point, end_point)
    p.add_mesh(line, color='red')


p.add_mesh_threshold(pcd_hood, 'linearity', title="Linearity", all_scalars=True, render_points_as_spheres=True, point_size=10)
p.show()

Widget(value='<iframe src="http://localhost:39901/index.html?ui=P_0x7f44440e7610_3&reconnect=auto" class="pyvi…